In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Modelagem de Valor de Vida do Cliente (Customer Lifetime Value - CLV)

A definição do CLV como o Valor Presente da Receita Esperada do cliente para os próximos 24 meses é a abordagem mais robusta e estratégica. Ela alinha a modelagem diretamente com a saúde financeira de longo prazo da empresa.

Para prever esse valor, migraremos para um problema de Regressão, onde a variável alvo será um valor monetário (CLV).

Próximo Passo: Engenharia de Variáveis para CLV (RFM)
Antes de construir o modelo de Regressão, precisamos enriquecer nosso conjunto de dados com as métricas que historicamente têm maior poder preditivo sobre o valor futuro do cliente.

A Análise RFM (Recência, Frequência, Valor Monetário) é o pilar desta fase. Para calcular o CLV futuro, o modelo precisa "aprender" com o comportamento passado do cliente.


In [ ]:
# Importando as bibliotecas essenciais
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    roc_curve,
    roc_auc_score,
    confusion_matrix,
    mean_squared_error
)

from xgboost import XGBClassifier, XGBRegressor

In [ ]:
# Variável de controle para reprodutibilidade
RANDOM_SEED = 42
pd.set_option('display.max_columns', None)

## Carregando Dataset

In [ ]:
clientes = pd.read_csv('/content/drive/MyDrive/01 - Organização Pessoal/Estudos /Estágio /clientes.csv')

clientes.head()

,Customer_ID,Age,Gender,Subscription_Length,Region,Payment_Method,Support_Tickets_Raised,Satisfaction_Score,Discount_Offered,Last_Activity,Monthly_Spend,Churned,score_churn
0,CUST000001,56.0,Male,54,South,PayPal,0,9.0,6.42,319,62.11,1,0.212832
1,CUST000002,69.0,Female,21,East,Debit Card,1,2.0,13.77,166,37.27,1,0.990108
2,CUST000003,46.0,Female,49,East,PayPal,3,8.0,19.91,207,61.82,0,0.188005
3,CUST000004,32.0,Male,47,West,Debit Card,3,1.0,13.39,108,40.96,1,0.990108
4,CUST000005,60.0,Male,6,East,Credit Card,2,NaN,13.18,65,45.97,0,0.213780


## Análise Exploratória

In [ ]:
clientes.shape

(5000, 13)

In [ ]:
clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer_ID             5000 non-null   object 
 1   Age                     4500 non-null   float64
 2   Gender                  5000 non-null   object 
 3   Subscription_Length     5000 non-null   int64  
 4   Region                  5000 non-null   object 
 5   Payment_Method          5000 non-null   object 
 6   Support_Tickets_Raised  5000 non-null   int64  
 7   Satisfaction_Score      4500 non-null   float64
 8   Discount_Offered        5000 non-null   float64
 9   Last_Activity           5000 non-null   int64  
 10  Monthly_Spend           5000 non-null   float64
 11  Churned                 5000 non-null   int64  
 12  score_churn             5000 non-null   float64
dtypes: float64(5), int64(4), object(4)
memory usage: 507.9+ KB


In [ ]:
clientes.describe(include='all')

,Customer_ID,Age,Gender,Subscription_Length,Region,Payment_Method,Support_Tickets_Raised,Satisfaction_Score,Discount_Offered,Last_Activity,Monthly_Spend,Churned,score_churn
count,5000,4500.000000,5000,5000.000000,5000,5000,5000.000000,4500.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
unique,5000,NaN,2,NaN,4,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,CUST005000,NaN,Female,NaN,West,Debit Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,2514,NaN,1316,1697,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,43.582222,NaN,29.704400,NaN,NaN,2.037000,5.54600,12.458404,181.441400,46.619784,0.448000,0.454540
std,NaN,14.969559,NaN,17.050336,NaN,NaN,1.422405,2.86929,4.325381,104.500951,9.522140,0.497338,0.353343
min,NaN,18.000000,NaN,1.000000,NaN,NaN,0.000000,1.00000,5.000000,1.000000,-5.060000,0.000000,0.167406
25%,NaN,31.000000,NaN,15.000000,NaN,NaN,1.000000,3.00000,8.700000,90.000000,39.897500,0.000000,0.212832
50%,NaN,43.000000,NaN,29.000000,NaN,NaN,2.000000,6.00000,12.500000,182.000000,46.625000,0.000000,0.229239
75%,NaN,56.000000,NaN,44.000000,NaN,NaN,3.000000,8.00000,16.190000,271.000000,53.210000,1.000000,0.990108


### Imputação de Valores Nulos usando a Mediana

In [ ]:
imputation_dict = {}
for col in ['Satisfaction_Score', 'Age']:
    if clientes[col].isnull().any():
          imputation_dict[col] = clientes[col].median()

if imputation_dict:
    clientes.fillna(imputation_dict, inplace=True)
    print(f"Imputação de NaN realizada com Mediana nas colunas: {list(imputation_dict.keys())}")
else:
    print("Nenhuma imputação necessária nas colunas esperadas.")

Imputação de NaN realizada com Mediana nas colunas: ['Satisfaction_Score', 'Age']


## Data Preparation para CLV

In [ ]:
clientes.columns

Index(['Customer_ID', 'Age', 'Gender', 'Subscription_Length', 'Region',
       'Payment_Method', 'Support_Tickets_Raised', 'Satisfaction_Score',
       'Discount_Offered', 'Last_Activity', 'Monthly_Spend', 'Churned',
       'score_churn'],
      dtype='object')

In [ ]:
clientes.drop(columns='Churned')

,Customer_ID,Age,Gender,Subscription_Length,Region,Payment_Method,Support_Tickets_Raised,Satisfaction_Score,Discount_Offered,Last_Activity,Monthly_Spend,score_churn
0,CUST000001,56.0,Male,54,South,PayPal,0,9.0,6.42,319,62.11,0.212832
1,CUST000002,69.0,Female,21,East,Debit Card,1,2.0,13.77,166,37.27,0.990108
2,CUST000003,46.0,Female,49,East,PayPal,3,8.0,19.91,207,61.82,0.188005
3,CUST000004,32.0,Male,47,West,Debit Card,3,1.0,13.39,108,40.96,0.990108
4,CUST000005,60.0,Male,6,East,Credit Card,2,6.0,13.18,65,45.97,0.213780
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,CUST004996,24.0,Male,5,East,Debit Card,0,8.0,18.03,121,51.61,0.208605
4996,CUST004997,66.0,Female,12,East,PayPal,3,5.0,5.84,192,36.32,0.241585
4997,CUST004998,26.0,Female,1,East,Credit Card,2,8.0,6.76,354,42.07,0.210851
4998,CUST004999,53.0,Female,44,South,Debit Card,1,10.0,5.95,31,60.09,0.212832


In [ ]:
transactions = clientes.copy()

# Monthly_Spend * 24  = Receita Bruta Esperada em 24 meses
# (1 - score_churn)   = Probabilidade de Retenção (1 - Probabilidade de Churn)

transactions['CLV_Target'] = transactions['Monthly_Spend'] * 24 * (1 - transactions['score_churn'])

# O novo DataFrame 'transactions' agora possui a variável alvo (Y)
transactions.head()

,Customer_ID,Age,Gender,Subscription_Length,Region,Payment_Method,Support_Tickets_Raised,Satisfaction_Score,Discount_Offered,Last_Activity,Monthly_Spend,Churned,score_churn,CLV_Target
0,CUST000001,56.0,Male,54,South,PayPal,0,9.0,6.42,319,62.11,1,0.212832,1173.383407
1,CUST000002,69.0,Female,21,East,Debit Card,1,2.0,13.77,166,37.27,1,0.990108,8.848080
2,CUST000003,46.0,Female,49,East,PayPal,3,8.0,19.91,207,61.82,0,0.188005,1204.740237
3,CUST000004,32.0,Male,47,West,Debit Card,3,1.0,13.39,108,40.96,1,0.990108,9.724104
4,CUST000005,60.0,Male,6,East,Credit Card,2,6.0,13.18,65,45.97,0,0.213780,867.421133


# Prepare Data

In [ ]:
# Colunas a serem removidas: CLV_Target (o Y), Monthly_Spend (Leakage), e identificadores (Customer_ID)

transactions = transactions.drop(columns=['Monthly_Spend', 'Customer_ID', 'Churned'])

### One-Hot Enconding

In [ ]:
transactions = pd.get_dummies(transactions)

In [ ]:
transactions.head()

,Age,Subscription_Length,Support_Tickets_Raised,Satisfaction_Score,Discount_Offered,Last_Activity,score_churn,CLV_Target,Gender_Female,Gender_Male,Region_East,Region_North,Region_South,Region_West,Payment_Method_Credit Card,Payment_Method_Debit Card,Payment_Method_PayPal
0,56.0,54,0,9.0,6.42,319,0.212832,1173.383407,False,True,False,False,True,False,False,False,True
1,69.0,21,1,2.0,13.77,166,0.990108,8.848080,True,False,True,False,False,False,False,True,False
2,46.0,49,3,8.0,19.91,207,0.188005,1204.740237,True,False,True,False,False,False,False,False,True
3,32.0,47,3,1.0,13.39,108,0.990108,9.724104,False,True,False,False,False,True,False,True,False
4,60.0,6,2,6.0,13.18,65,0.213780,867.421133,False,True,True,False,False,False,True,False,False


In [ ]:
# 1. Definição de X e Y
X = transactions.drop(columns=['CLV_Target'])
y = transactions['CLV_Target']

In [ ]:
X.columns

Index(['Age', 'Subscription_Length', 'Support_Tickets_Raised',
       'Satisfaction_Score', 'Discount_Offered', 'Last_Activity',
       'score_churn', 'Gender_Female', 'Gender_Male', 'Region_East',
       'Region_North', 'Region_South', 'Region_West',
       'Payment_Method_Credit Card', 'Payment_Method_Debit Card',
       'Payment_Method_PayPal'],
      dtype='object')

In [ ]:
# 2. Split de Dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
# 3. Criação do Pipeline de Regressão
pipeline_xgb_reg = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb_reg', XGBRegressor(
        objective='reg:squarederror', # Objetivo para Regressão: Mínimos Quadrados
        n_estimators=100,
        random_state=42
    ))
])

In [ ]:
# 4. Treinamento
pipeline_xgb_reg.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('xgb_reg',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=100, n_jobs=None,
                              num_parallel_tree=None, ...))])

In [ ]:
# 5. Previsão e Avaliação (RMSE)
y_pred = pipeline_xgb_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
# 6. Avaliação (R-squared) - Quanto mais próximo de 1, melhor
r2_score = pipeline_xgb_reg.score(X_test, y_test)

In [ ]:
print(f"Resultado do Modelo de Regressão CLV:")
print(f"RMSE (Root Mean Squared Error): {rmse:.2f}")
print(f"R² Score: {r2_score:.4f}")

Resultado do Modelo de Regressão CLV:
RMSE (Root Mean Squared Error): 71.65
R² Score: 0.9746
